# Store cleaned data in Oracle

In [6]:
import oracledb
import pandas as pd

# Connect to Oracle (update credentials)
connection = oracledb.connect(
    user="system",
    password="root",
    dsn="localhost:1521/XE"
)
cursor = connection.cursor()

# Load your CSV
df = pd.read_csv("Dataset/ETHbanks_mobileApp_reviews_clean.csv")  # Update path
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
# Then use TO_DATE(:date_value, 'YYYY-MM-DD') in SQL
# Insert data
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews (
            bank_id, review_text, rating, review_date, source
        ) VALUES (
            (SELECT bank_id FROM banks WHERE bank_name = :bank_name),
            :review_text, :rating_value, TO_DATE(:date_value, 'YYYY-MM-DD'), :source_name
        )
        """,
        bank_name=row['bank'],
        review_text=row['review'],
        rating_value=row['rating'],
        date_value=row['date'],
        source_name=row['source']
    )

connection.commit()
print(f"Inserted {len(df)} records")
cursor.close()
connection.close()

Inserted 6817 records
